In [32]:
import dash 
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd 
import plotly.graph_objects as go 
import sys 

sys.path.append('../')

import omdutils as omd
import base64

In [4]:
# df = pd.read_csv('../Data/wheels.csv')
db = '../SourceData/datasets.db'
# omd.to_sqlite(df, 'wheels', db)
sql = 'SELECT * FROM wheels'
df = omd.from_sqlite(sql, db)
df.head()

,wheels,color,image
0,1,red,redunicycle.jpg
1,1,blue,blueunicycle.jpg
2,1,yellow,yellowunicycle.jpg
3,2,red,redbicycle.jpg
4,2,blue,bluemotorcycle.jpg


In [36]:
app = dash.Dash()

def encode_image(image_file):
    encoded = base64.b64encode(open(image_file, 'rb').read())
    return 'data:image/png;base64,{}'.format(encoded.decode())

app.layout = html.Div([
    html.H1('Web Application Dashboard'),
    html.H2('Display image based on dropdown selections'),

    html.Div([
        dcc.RadioItems(
            id='wheels',
            options=[{'label': i, 'value': i} for i in df['wheels'].unique()],
            value=1,
            labelStyle={'display': 'inline-block'}
        ),
        html.Div(id='wheels-output'),
        html.Hr(),
        dcc.RadioItems(
            id='colors',
            options=[{'label': i, 'value': i} for i in df['color'].unique()],
            value='blue',
            labelStyle={'display': 'inline-block'}
        ),
        html.Div(id='colors-output'),
        html.Img(id='display-image', src='children', height=300)
    ]),
], style={'fontFamily': 'helvetica', 'fontSize': 18})


@app.callback(Output('wheels-output', 'children'),
              Input('wheels', 'value'))
def callback_a(wheels_value):
    return f'You chose {wheels_value}'


@app.callback(Output('colors-output', 'children'),
              Input('colors', 'value'))
def callback_b(colors_value):
    return f'You chose {colors_value}'


@app.callback(Output('display-image', 'src'),
              Input('wheels', 'value'),
              Input('colors', 'value'))
def callback_image(wheels, colors):
    path = '../Data/Images/'
    return encode_image(path + df[(df ['wheels'] == wheels) & (df['color'] == colors)]['image'].values[0])

app.run(debug=True)